In [17]:
# Libaries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt

from imblearn.over_sampling import SMOTE

import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
numerical = pd.read_csv(r"C:\Users\ljant\Desktop\Ironhack\16_Machine_learning_advanced\lab-random-forests\files_for_lab\numerical.csv")
targets = pd.read_csv(r"C:\Users\ljant\Desktop\Ironhack\16_Machine_learning_advanced\lab-random-forests\files_for_lab\target.csv")
categorical = pd.read_csv(r"C:\Users\ljant\Desktop\Ironhack\16_Machine_learning_advanced\lab-random-forests\files_for_lab\categorical.csv")

In [3]:
df = pd.concat([numerical, categorical, targets], axis = 1)
df

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,...,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,TARGET_B,TARGET_D
0,0,60.000000,5,9,0,0,39,34,18,10,...,92,8,94,2,95,12,89,11,0,0.0
1,1,46.000000,6,9,16,0,15,55,11,6,...,93,10,95,12,95,12,93,10,0,0.0
2,1,61.611649,3,1,2,0,20,29,33,6,...,91,11,92,7,95,12,90,1,0,0.0
3,0,70.000000,1,4,2,0,23,14,31,3,...,87,11,94,11,95,12,87,2,0,0.0
4,0,78.000000,3,2,60,1,28,9,53,26,...,93,10,96,1,96,1,79,3,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,1,61.611649,5,9,0,14,36,47,11,7,...,96,2,96,2,96,2,96,2,0,0.0
95408,1,48.000000,7,9,1,0,31,43,19,4,...,96,3,96,3,96,3,96,3,0,0.0
95409,1,60.000000,5,9,0,0,18,46,20,7,...,96,3,95,1,96,10,94,10,0,0.0
95410,0,58.000000,7,9,0,0,28,35,20,9,...,90,11,96,8,97,1,86,12,1,18.0


In [4]:
def header_function (data):
    cols = []
    for i in range (len(data.columns)):
        cols.append(data.columns[i].lower().replace(' ','_'))
    
    data.columns = cols

    return data

header_function(df)

,tcode,age,income,wealth1,hit,malemili,malevet,vietvets,wwiivets,localgov,...,minrdate_yr,minrdate_mm,maxrdate_yr,maxrdate_mm,lastdate_yr,lastdate_mm,firstdate_yr,firstdate_mm,target_b,target_d
0,0,60.000000,5,9,0,0,39,34,18,10,...,92,8,94,2,95,12,89,11,0,0.0
1,1,46.000000,6,9,16,0,15,55,11,6,...,93,10,95,12,95,12,93,10,0,0.0
2,1,61.611649,3,1,2,0,20,29,33,6,...,91,11,92,7,95,12,90,1,0,0.0
3,0,70.000000,1,4,2,0,23,14,31,3,...,87,11,94,11,95,12,87,2,0,0.0
4,0,78.000000,3,2,60,1,28,9,53,26,...,93,10,96,1,96,1,79,3,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,1,61.611649,5,9,0,14,36,47,11,7,...,96,2,96,2,96,2,96,2,0,0.0
95408,1,48.000000,7,9,1,0,31,43,19,4,...,96,3,96,3,96,3,96,3,0,0.0
95409,1,60.000000,5,9,0,0,18,46,20,7,...,96,3,95,1,96,10,94,10,0,0.0
95410,0,58.000000,7,9,0,0,28,35,20,9,...,90,11,96,8,97,1,86,12,1,18.0


In [6]:
df = df.drop_duplicates()

In [7]:
df = df.dropna()

In [8]:
# Cleaning data for model

X = df.drop(['target_b', 'target_d'], axis = 1)
Y_b = df['target_b']
Y_d = df['target_d']

numericalX = X.select_dtypes('number').reset_index(drop = True)
categorcalX = X.select_dtypes('object').reset_index(drop = True)
categorcalX = categorcalX.drop(['state'], axis = 1)
categorcalX = pd.get_dummies(categorcalX[['homeownr', 'gender', 'rfa_2r', 'rfa_2a', 'geocode2', 'domain_a']], dtype = int)

X = pd.concat([numericalX, categorcalX], axis = 1)

# Train-Test-Split

In [11]:
X_train, X_test, Y_train_b, Y_test_b, Y_train_d, Y_test_d = train_test_split(X, Y_b,Y_d, test_size=0.2, random_state=0)
train = pd.concat([X_train.reset_index(drop=True), 
                   Y_train_b.reset_index(drop=True), 
                   Y_train_d.reset_index(drop=True)], axis=1)

# Optimising Model

In [12]:
X_train.reset_index(drop=True, inplace=True)
Y_train_b.reset_index(drop=True, inplace=True)

def significant_features(X, Y):
    """
    Select independent variables with P>|z| < 0.05 from the Logit model summary.
    """
    model = sm.Logit(Y_train_b, X_train).fit()
    summary = model.summary()

    significant_features = []
    table = summary.tables[1]
    headers = table.data[0]
    data = table.data[1:]

    p_value_index = headers.index('P>|z|') if 'P>|z|' in headers else headers.index('P>|z| [0.025')

    for row in data:
        p_value = float(row[p_value_index])
        if p_value < 0.05:
            significant_features.append(row[0])

    return significant_features

relevant_features = significant_features(X_train, Y_train_b)
print("Independent variables with P>|t| < 0.05:", relevant_features)

Optimization terminated successfully.
         Current function value: 0.192267
         Iterations 9
Independent variables with P>|t| < 0.05: ['age', 'income', 'wealth1', 'hhn3', 'hu3', 'hu4', 'rhp2', 'dma', 'ic11', 'tpe7', 'lfc2', 'lfc4', 'occ6', 'ec1', 'voc3', 'cardprom', 'numprm12', 'ngiftall', 'cardgift', 'timelag', 'rfa_2f', 'domain_b', 'odatew_mm', 'dob_mm', 'lastdate_yr', 'lastdate_mm', 'firstdate_yr']


In [13]:
X_train_adj = X_train[relevant_features]
X_test_adj = X_test[relevant_features]

model = LogisticRegression()
model.fit(X_train_adj, Y_train_b)

print(model.score(X_train_adj, Y_train_b))
print(model.score(X_test_adj, Y_test_b))

0.9496521636599458
0.9475973379447676


# Dealing with Unbalance - SMOTE

In [14]:
train = pd.concat([X_train_adj.reset_index(drop=True), 
                   Y_train_b.reset_index(drop=True), 
                   Y_train_d.reset_index(drop=True)], axis=1)
print(train['target_b'].value_counts())

0    72486
1     3843
Name: target_b, dtype: int64


In [20]:
X = train.drop(['target_b', 'target_d'], axis=1)
Y = train['target_b']

# Using SMOTE to oversample the minority class
smote = SMOTE(random_state=0, sampling_strategy=1.0)
X_resampled, Y_resampled = smote.fit_resample(X, Y)

# Combining the resampled features and target into a new DataFrame
train_smote = pd.concat([X_resampled, Y_resampled], axis=1)

# Check for NaNs after SMOTE
assert not train_smote.isnull().any().any(), "NaNs introduced during resampling"

# Print the value counts for the target variable
print(train_smote['target_b'].value_counts())

0    72486
1    72486
Name: target_b, dtype: int64


# Random Forest Model

In [23]:
# Separating the features and target variables
Y_train_smote_b = train_smote['target_b']
X_train_smote = train_smote.drop(['target_b'], axis=1)

In [26]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators = 50, max_depth = 7, random_state = 42)
forest.fit(X_train_smote, Y_train_smote_b)

print(forest.score(X_train_smote, Y_train_smote_b))
forest.score(X_test_adj, Y_test_b)

0.7502000386281489


0.6898810459571346

# Hyperparameter Tuning

In [27]:
from sklearn.model_selection import GridSearchCV

# 1. Define parameter grid
param_grid = {'n_estimators' : [50, 100, 200],
              'max_depth' : [2, 4, 6],
              'min_samples_split' : [2, 4],
              'min_samples_leaf' : [1,2] 
}

# 2. Define which model to test
forest = RandomForestClassifier(random_state = 42)

grid_search = GridSearchCV(forest, param_grid, cv = 5, return_train_score = True, n_jobs = -1)
grid_search.fit(X_train_smote, Y_train_smote_b)

grid_search.best_params_

{'max_depth': 6,
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 100}

In [29]:
# Adjusted model

forest = RandomForestClassifier(n_estimators = 100, max_depth = 6, random_state = 42, min_samples_leaf = 2, min_samples_split = 2)
forest.fit(X_train_smote, Y_train_smote_b)

print(forest.score(X_train_smote, Y_train_smote_b))
forest.score(X_test_adj, Y_test_b)

0.7354316695637778


0.6681339412042132